In [1]:
# To make data frames
import pandas as pd

# To list files in a directory
import os
# Move up in directories
print(os.getcwd()) # for debugging
os.chdir("..")
print(os.getcwd()) # for debugging

# To work with .json files
import json

# To parse dates
from datetime import datetime

C:\Users\leyjf\Dropbox\github\FitbitStatsProject\CleanedData
C:\Users\leyjf\Dropbox\github\FitbitStatsProject


# Exercise

In [5]:
# List of files using os
raw_exercise_files_path = "RawData\RawExerciseFiles"
raw_exercise_files_list = os.listdir(raw_exercise_files_path)
#print(raw_exercise_files_list) # for debugging        

all_extracted_data = []
for file in raw_exercise_files_list:
    # with is a more efficient way of reading files
    with open(raw_exercise_files_path+"\\"+file) as file:
        parsed_file = json.load(file)
    # Extract the data I want
    for entry in parsed_file:
        #print(entry) # for debugging
        if entry["activityName"]!="Weights":
            if entry["averageHeartRate"]>134: 
                activityName = "Run"
            else: 
                activityName = "Walk"
            extracted_data = {
                "activityName" : activityName,
                "averageHeartRate" : entry["averageHeartRate"],
                "belowZonesMinutes": entry["heartRateZones"][0]["minutes"],
                "fatBurnMinutes": entry["heartRateZones"][1]["minutes"],
                "cardioMinutes": entry["heartRateZones"][2]["minutes"],
                "peakMinutes" : entry["heartRateZones"][3]["minutes"],
                "startTime": entry["startTime"],
                "sessionMinutes": entry["heartRateZones"][0]["minutes"]+ 
                                     entry["heartRateZones"][1]["minutes"]+
                                     entry["heartRateZones"][2]["minutes"]+
                                    entry["heartRateZones"][3]["minutes"]
                } 
            # Append the list
            all_extracted_data.append(extracted_data)
#print(type(all_extracted_data)) # for debugging
#print(all_extracted_data) # for debugging
# Change to a json string    
all_extracted_data_string = json.dumps(all_extracted_data)
#print(all_extracted_data_string) # for debugging
all_extracted_data_json = json.loads(all_extracted_data_string)
#print(all_extracted_data_json) # for debugging

# Now group the extracted data by date
grouped_by_date = []
for i in range(len(all_extracted_data)):
    #print(all_extracted_data[i]) # for debugging
    date = all_extracted_data[i]["startTime"].split()[0]
    #print(date) # for debugging
    if i == 0 or date != all_extracted_data[i-1]["startTime"].split()[0]:
        j = 1
        exercise = {
            "date" : date,
            "exercise"+str(j) : all_extracted_data[i]
        }
        j = j + 1
    else: 
        exercise["exercise"+str(j)] = all_extracted_data[i]
        j = j + 1
    if i < len(all_extracted_data)-1 and date != all_extracted_data[i+1]["startTime"].split()[0]:    
        #print(date) # for debugging
        #print(all_extracted_data[i+1]["startTime"].split()[0]) # for debugging
        grouped_by_date.append(exercise)
    elif i == len(all_extracted_data)-1:
        grouped_by_date.append(exercise)
#print(grouped_by_date) # for debugging    

# Group exercise sessions into one array
new_grouped_by_date = []
for day in grouped_by_date:
    new_day = {
        "date": day["date"],
        "exercise": [day["exercise"+str(i)] for i in range(1,len(day))]    
    }
    new_grouped_by_date.append(new_day)
#print(new_grouped_by_date) # for debugging

# Total exercise per day
for day in new_grouped_by_date:
    total_minutes = 0
    for i in range(len(day["exercise"])):
        total_minutes = sum((day["exercise"][i]["belowZonesMinutes"],
                            day["exercise"][i]["fatBurnMinutes"],
                            day["exercise"][i]["cardioMinutes"],
                            day["exercise"][i]["peakMinutes"]),
                            total_minutes)
    day["totalMinutes"] = total_minutes    
    #print(day) # for debugging

# Accumulated exercise
for day in new_grouped_by_date:
    for i in range(len(day["exercise"])):
        accumulated_minutes = 0
        for j in range(i):
            accumulated_minutes = accumulated_minutes + day["exercise"][j]["sessionMinutes"]
        day["exercise"][i]["accumulatedMinutes"] = accumulated_minutes    

# Month for each day
for day in new_grouped_by_date:
    month_year = datetime.strptime(day["date"], "%m/%d/%y").strftime('%B %Y')
    day["month"] = month_year
    #print(day) # for debugging
    
# Save parsed string to a .json file
new_grouped_by_date_string = json.dumps(new_grouped_by_date)
new_grouped_by_date_json = json.loads(new_grouped_by_date_string)
save_file = open("CleanedData\\exercise_new_grouped_by_date.json", "w")  
json.dump(new_grouped_by_date_json, save_file, indent = 4)  
save_file.close()

# Resting heart rate

In [ ]:
# List of files using os
raw_heart_rate_files_path = "RawData\RawHeartRateFiles"
raw_heart_rate_files_list = os.listdir(raw_heart_rate_files_path)
 
resting_heart_rate_data = []
# Can't use list comprehension with extend function(?)
for file in raw_heart_rate_files_list:
    # with is a more efficient way of reading files
    with open(raw_heart_rate_files_path+"\\"+file) as file:
        parsed_file = json.load(file)
    resting_hr = min([entry["value"]["bpm"] for entry in parsed_file])    
    #print("Resting hr is "+str(resting_hr)+".") # for debugging
    new_dict = {
        "date" : parsed_file[0]["dateTime"].split()[0],
        "resting_hr" : resting_hr
        }
    #print(new_dict) # for debugging
    resting_heart_rate_data.append(new_dict)    

# Change to a json string    
resting_heart_rate_data_string = json.dumps(resting_heart_rate_data)

# Save parsed string to a .json file
save_file = open("CleanedData\\resting_hr_cleaned.json", "w")  
json.dump(json.loads(resting_heart_rate_data_string), save_file, indent = 4)  
save_file.close()